In [0]:
import pandas as pd
import numpy as np

In [0]:
pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144795 sha256=5ca3caa372292c29c5909b962577b4af66fe744aefcfee5daabba6931e517a8d
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [0]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import string

In [0]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
os.chdir('/content/gdrive/My Drive/Analytics in Financial Services Project/Cleaning by-products')

In [0]:
os.getcwd()

'/content/gdrive/My Drive/Analytics in Financial Services Project'

In [0]:
demo = pd.read_csv('Demography new.csv')

In [0]:
trans = pd.read_csv('transaction.csv')

In [0]:
demo.head()

,No,AgeBand,CreditLimitBand,District,Gender,PartyID,RevolverFlag
0,1,01 (20 - 29),01 (< 10)K,53,F,10167887,06 Inactive
1,2,01 (20 - 29),01 (< 10)K,56,F,35173089,02 Transactor
2,3,01 (20 - 29),01 (< 10)K,15,F,86406397,01 New Customer
3,4,01 (20 - 29),01 (< 10)K,73,F,13198251,01 New Customer
4,5,01 (20 - 29),01 (< 10)K,67,F,2074308,02 Transactor


In [0]:
trans.head()

,PartyID,CardType,Merchant_Category,Merchant_Name,Transaction_amount,Transaction_Date,Unnamed: 6
0,10100028,Co-Brand,FOOD STORES-WAREHOUSE,FAIRPRICE XTRA-AMK SINGAPORE SG,33.00,01/03/2014,NaN
1,10104387,Co-Brand,DEPARTMENT STORES,ROBINSONS - RAFFLES CITY SINGAPORE SG,122.21,01/03/2014,NaN
2,10124427,Co-Brand,DEPARTMENT STORES,M & S - PARAGON SHPG CTR SINGAPORE SG,53.33,01/03/2014,NaN
3,10128840,Core Card,OTHER SERVICES,EXTREMAL SHOP ALMATY KAZ,15.70,01/03/2014,NaN
4,10199312,Core Card,RESTAURANTS-BARS,KYO SINGAPORE SG,133.32,01/03/2014,NaN


In [0]:
merged = pd.merge(demo,trans,on='PartyID',how = 'left')

In [0]:
merged.head()

,No,AgeBand,CreditLimitBand,District,Gender,PartyID,RevolverFlag,CardType,Merchant_Category,Merchant_Name,Transaction_amount,Transaction_Date,Unnamed: 6
0,1,01 (20 - 29),01 (< 10)K,53,F,10167887,06 Inactive,NaN,NaN,NaN,NaN,NaN,NaN
1,2,01 (20 - 29),01 (< 10)K,56,F,35173089,02 Transactor,Core Card,RESTAURANTS-BARS,IKEA-RESTAURANT SINGAPORE SG,32.44,09/03/2014,NaN
2,2,01 (20 - 29),01 (< 10)K,56,F,35173089,02 Transactor,Core Card,INTERIOR FURNISHINGS,IKEA SINGAPORE SG,26.78,09/03/2014,NaN
3,2,01 (20 - 29),01 (< 10)K,56,F,35173089,02 Transactor,Core Card,RESTAURANTS-BARS,PARIS BAGUETTE SINGAPORE SGP,26.66,28/03/2014,NaN
4,2,01 (20 - 29),01 (< 10)K,56,F,35173089,02 Transactor,Core Card,RESTAURANTS-BARS,PARIS BAGUETTE SINGAPORE SGP,53.55,28/03/2014,NaN


In [0]:
demo.shape, trans.shape, merged.shape

((29999, 7), (374035, 7), (381452, 13))

In [0]:
#Increase in transction count post merging - 7417 out of 29999 people with no transactions
print("Difference in transaction",381452-374035)
print(len(merged.loc[merged['CardType'].isnull(),'PartyID'].unique()))
print(100*len(merged.loc[merged['CardType'].isnull(),'PartyID'].unique())/len(demo)," percent customers with missing transactions")
print(100*len(merged.loc[merged['CardType'].isnull(),'PartyID'].unique())/len(merged)," percent transactions aggdf in merged aggdfframe")


Difference in transaction 7417
7417
24.72415747191573  percent customers with missing transactions
1.9444124031332908  percent transactions aggdf in merged aggdfframe


In [0]:
merged.dtypes

No                      int64
AgeBand                object
CreditLimitBand        object
District               object
Gender                 object
PartyID                 int64
RevolverFlag           object
CardType               object
Merchant_Category      object
Merchant_Name          object
Transaction_amount    float64
Transaction_Date       object
Unnamed: 6            float64
dtype: object

In [0]:
#missing dropped aggdfframe
mdmerged = merged.drop(axis = 0, index = merged[merged['CardType'].isnull()].index)

In [0]:
mdmerged['Transaction_Date'] = pd.to_datetime(mdmerged['Transaction_Date'],format="%d/%m/%Y")

In [0]:
mdmerged['Transaction_Date'].describe()

count                  374035
unique                     92
top       2014-05-31 00:00:00
freq                     5598
first     2014-03-01 00:00:00
last      2014-05-31 00:00:00
Name: Transaction_Date, dtype: object

In [0]:
for col in mdmerged.columns:
  if (merged[col].dtype == "object"):
    print("-----------------------------")
    print(100*mdmerged[col].value_counts()/len(mdmerged))
    print("Number of categories in",col,"is: ", len(mdmerged[col].value_counts()))
    print("-----------------------------")

-----------------------------
03 (40 - 49)    31.338778
02 (30 - 39)    28.817089
04 (50 - 59)    22.247116
05 (60 - 69)     8.953975
01 (20 - 29)     7.325250
06 (>= 70)       1.317791
Name: AgeBand, dtype: float64
Number of categories in AgeBand is:  6
-----------------------------
-----------------------------
01 (< 10)K         57.922120
02 (10 - 20)K      29.931958
03 (20 - 30)K       5.637708
04 (30 - 40)K       2.287486
06 (50 - 60)K       1.150427
05 (40 - 50)K       0.957397
14 (unknown)        0.561445
07 (60 - 70)K       0.371356
08 (70 - 80)K       0.370554
11 (10 - 150)K      0.349165
09 (80 - 90)K       0.229390
10 (90 - 100)K      0.130469
12 (150 - 200)K     0.061759
13 (>= 200)K        0.038766
Name: CreditLimitBand, dtype: float64
Number of categories in CreditLimitBand is:  14
-----------------------------
-----------------------------
52    5.844640
54    5.477562
53    5.265283
64    4.995789
68    4.357346
        ...   
49    0.017645
72    0.005347
90    0.00427

In [0]:
mdmerged.head()

,No,AgeBand,CreditLimitBand,District,Gender,PartyID,RevolverFlag,CardType,Merchant_Category,Merchant_Name,Transaction_amount,Transaction_Date,Unnamed: 6
1,2,01 (20 - 29),01 (< 10)K,56,F,35173089,02 Transactor,Core Card,RESTAURANTS-BARS,IKEA-RESTAURANT SINGAPORE SG,32.44,2014-03-09,NaN
2,2,01 (20 - 29),01 (< 10)K,56,F,35173089,02 Transactor,Core Card,INTERIOR FURNISHINGS,IKEA SINGAPORE SG,26.78,2014-03-09,NaN
3,2,01 (20 - 29),01 (< 10)K,56,F,35173089,02 Transactor,Core Card,RESTAURANTS-BARS,PARIS BAGUETTE SINGAPORE SGP,26.66,2014-03-28,NaN
4,2,01 (20 - 29),01 (< 10)K,56,F,35173089,02 Transactor,Core Card,RESTAURANTS-BARS,PARIS BAGUETTE SINGAPORE SGP,53.55,2014-03-28,NaN
5,3,01 (20 - 29),01 (< 10)K,15,F,86406397,01 New Customer,Core Card,PROFESSIONAL SERVICES,ADDON SYSTEMS PTE LTD SINGAPORE SG,2165.34,2014-05-18,NaN


In [0]:
aggdf= mdmerged.groupby('PartyID').aggregate({'No':'count','CardType':'nunique','Merchant_Category':'nunique','Merchant_Name':'nunique','Transaction_amount':['mean','sum'],'Transaction_Date':'max'})
aggdf.columns = ["_".join(x) for x in aggdf.columns.ravel()]

In [0]:
aggdf.head()

,No_count,CardType_nunique,Merchant_Category_nunique,Merchant_Name_nunique,Transaction_amount_mean,Transaction_amount_sum,Transaction_Date_max
PartyID,,,,,,,
174884,56,1,12,27,152.070893,8515.97,2014-05-31
174930,13,1,4,10,39.124615,508.62,2014-05-16
175016,13,1,5,8,30.654615,398.51,2014-04-29
175067,15,2,3,6,66.691333,1000.37,2014-05-29
175187,8,2,3,7,37.461250,299.69,2014-05-12


In [0]:
aggdf['DaysSinceLastPurchase'] = (pd.to_datetime("2014-05-31",format="%Y-%m-%d")-aggdf['Transaction_Date_max'])/pd.Timedelta(1,'d')

In [0]:
#Recency Variable

aggdf['DaysSinceLastPurchase_Rev'] = aggdf['DaysSinceLastPurchase'].apply(lambda x: -1*x)

print(aggdf['DaysSinceLastPurchase_Rev'].describe())

print('\n\n',aggdf['DaysSinceLastPurchase_Rev'].quantile([0,0.1,0.2,0.3,0.35,0.4,0.5,0.6,0.7,0.75,0.8,0.90,0.99,1]))

aggdf['R_quantile'] = pd.qcut(aggdf['DaysSinceLastPurchase_Rev'], 5 ,['1','2','3','4','5'], duplicates='drop')

print('\n\n',aggdf['R_quantile'].value_counts())

count    22582.000000
mean       -14.106279
std         19.552700
min        -91.000000
25%        -19.000000
50%         -6.000000
75%         -1.000000
max         -0.000000
Name: DaysSinceLastPurchase_Rev, dtype: float64


 0.00   -91.0
0.10   -43.0
0.20   -24.0
0.30   -15.0
0.35   -12.0
0.40    -9.0
0.50    -6.0
0.60    -3.0
0.70    -2.0
0.75    -1.0
0.80    -1.0
0.90    -0.0
0.99    -0.0
1.00    -0.0
Name: DaysSinceLastPurchase_Rev, dtype: float64


 3    5516
1    4628
2    4512
5    4097
4    3829
Name: R_quantile, dtype: int64


In [0]:
#Verifying that Recency bin 5 indeed consists of customers who recently purchased

aggdf.loc[aggdf['R_quantile']=='5','DaysSinceLastPurchase'].mean()

0.0

In [0]:
#Frequency Variable

print(aggdf['No_count'].describe())

print('\n\n',aggdf['No_count'].quantile([0,0.1,0.2,0.3,0.35,0.4,0.5,0.6,0.7,0.75,0.8,0.90,0.99,1]))

aggdf['F_quantile'] = pd.qcut(aggdf['No_count'], 5 ,['1','2','3','4','5'], duplicates='drop')

print('\n\n',aggdf['F_quantile'].value_counts())

count    22582.000000
mean        16.563413
std         21.936885
min          1.000000
25%          4.000000
50%         10.000000
75%         22.000000
max       1350.000000
Name: No_count, dtype: float64


 0.00       1.0
0.10       1.0
0.20       3.0
0.30       5.0
0.35       6.0
0.40       7.0
0.50      10.0
0.60      14.0
0.70      19.0
0.75      22.0
0.80      27.0
0.90      39.0
0.99      89.0
1.00    1350.0
Name: No_count, dtype: float64


 1    5453
4    4478
3    4304
5    4257
2    4090
Name: F_quantile, dtype: int64


In [0]:
#Monetary Variable

print(aggdf['Transaction_amount_sum'].describe())
print('\n\n',aggdf['Transaction_amount_sum'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.90,0.99,1]))

aggdf['M_quantile'] = pd.qcut(aggdf['Transaction_amount_sum'], 5 ,['1','2','3','4','5'], duplicates = 'drop')

print('\n\n',aggdf['M_quantile'].value_counts())

count    2.258200e+04
mean     2.231907e+03
std      2.124770e+04
min      6.700000e-01
25%      3.057225e+02
50%      8.879850e+02
75%      2.276950e+03
max      2.594885e+06
Name: Transaction_amount_sum, dtype: float64


 0.10    9.932000e+01
0.20    2.233120e+02
0.30    3.950390e+02
0.40    6.068460e+02
0.50    8.879850e+02
0.60    1.284672e+03
0.70    1.864757e+03
0.80    2.830418e+03
0.90    4.825187e+03
0.99    1.593825e+04
1.00    2.594885e+06
Name: Transaction_amount_sum, dtype: float64


 5    4517
1    4517
4    4516
3    4516
2    4516
Name: M_quantile, dtype: int64


In [0]:
#RFM Variable

aggdf['RFM_q'] = aggdf[['R_quantile','F_quantile','M_quantile']].apply(lambda x: ''.join(x),axis=1)
aggdf['RFM_q'].value_counts()

111    2089
555    1113
455     786
211     702
344     595
       ... 
153       2
252       2
141       1
351       1
251       1
Name: RFM_q, Length: 121, dtype: int64

In [0]:
#EngagementScore
aggdf['EngagementScore'] = aggdf[['R_quantile','F_quantile','M_quantile']].apply(lambda x: int(x['R_quantile'])+int(x['F_quantile'])+int(x['M_quantile']),axis=1)

In [0]:
#Identifying EngagementSegments
print(aggdf['EngagementScore'].describe())
print('\n\n',aggdf['EngagementScore'].quantile([0.0,0.33,0.66,0.99,1]))

aggdf['EngagementSegments'] = pd.qcut(aggdf['EngagementScore'],3,['Low','Mid','High'], duplicates = 'drop')

print('\n\n',aggdf['EngagementSegments'].value_counts())

count    22582.000000
mean         8.833983
std          3.627187
min          3.000000
25%          6.000000
50%          9.000000
75%         12.000000
max         15.000000
Name: EngagementScore, dtype: float64


 0.00     3.0
0.33     7.0
0.66    11.0
0.99    15.0
1.00    15.0
Name: EngagementScore, dtype: float64


 Low     8671
Mid     7497
High    6414
Name: EngagementSegments, dtype: int64


In [0]:
#Characteristics of high-mid-low segments
for seg in aggdf['EngagementSegments'].unique():
  print("----------------------")
  print("Segment:",seg)
  print("EngagementScore Range- min, mean, max: ", aggdf.loc[aggdf['EngagementSegments']==seg,'EngagementScore'].min(),aggdf.loc[aggdf['EngagementSegments']==seg,'EngagementScore'].mean(),aggdf.loc[aggdf['EngagementSegments']==seg,'EngagementScore'].max())
  print("Quantile means: Rq_mean, Fq_mean, Mq_mean: ", pd.to_numeric(aggdf.loc[aggdf['EngagementSegments']==seg,'R_quantile']).mean(),pd.to_numeric(aggdf.loc[aggdf['EngagementSegments']==seg,'F_quantile']).mean(),pd.to_numeric(aggdf.loc[aggdf['EngagementSegments']==seg,'M_quantile']).mean())
  print("Quantile min: Rq_min, Fq_min, Mq_min: ", pd.to_numeric(aggdf.loc[aggdf['EngagementSegments']==seg,'R_quantile']).min(),pd.to_numeric(aggdf.loc[aggdf['EngagementSegments']==seg,'F_quantile']).min(),pd.to_numeric(aggdf.loc[aggdf['EngagementSegments']==seg,'M_quantile']).min())
  print("Quantile max: Rq_max, Fq_max, Mq_max: ", pd.to_numeric(aggdf.loc[aggdf['EngagementSegments']==seg,'R_quantile']).max(),pd.to_numeric(aggdf.loc[aggdf['EngagementSegments']==seg,'F_quantile']).max(),pd.to_numeric(aggdf.loc[aggdf['EngagementSegments']==seg,'M_quantile']).max())
  print("Value means: R_mean, F_mean, M_mean: ",aggdf.loc[aggdf['EngagementSegments']==seg,'DaysSinceLastPurchase'].mean(),aggdf.loc[aggdf['EngagementSegments']==seg,'No_count'].mean(),aggdf.loc[aggdf['EngagementSegments']==seg,'Transaction_amount_sum'].mean())
  print("Unique RFM Segments: ", aggdf.loc[aggdf['EngagementSegments']==seg,'RFM_q'].unique())
  print("----------------------")

----------------------
Segment: High
EngagementScore Range- min, mean, max:  12 13.290926099158092 15
Quantile means: Rq_mean, Fq_mean, Mq_mean:  4.262394761459308 4.575304022450888 4.453227315247895
Quantile min: Rq_min, Fq_min, Mq_min:  2 2 2
Quantile max: Rq_max, Fq_max, Mq_max:  5 5 5
Value means: R_mean, F_mean, M_mean:  1.4998440910508264 37.86295603367633 5428.334298409732
Unique RFM Segments:  ['555' '544' '454' '445' '554' '455' '453' '355' '553' '535' '444' '545'
 '543' '345' '525' '354' '255' '534' '435' '552']
----------------------
----------------------
Segment: Low
EngagementScore Range- min, mean, max:  3 4.952715949717449 7
Quantile means: Rq_mean, Fq_mean, Mq_mean:  1.76311844077961 1.4754930227194096 1.7141044862184291
Quantile min: Rq_min, Fq_min, Mq_min:  1 1 1
Quantile max: Rq_max, Fq_max, Mq_max:  5 4 5
Value means: R_mean, F_mean, M_mean:  29.42290393264906 3.5452658286241494 403.7756498673737
Unique RFM Segments:  ['232' '132' '222' '122' '311' '213' '112' '111

In [0]:
#aggdf.to_excel('SegmentedDf.xlsx',index=True)

## Loading cleaned dataset and cleaning merchant names further

> Indented block



In [0]:
Trx_Cleaned = pd.read_excel('Transactions_Cleaned.xlsx')

In [0]:
Demographics_Cleaned = pd.read_excel('Demography_Cleaned.xlsx') 

In [0]:
Trx_Cleaned['Merchant_Name_Cleaned'].nunique()

31147

In [0]:
Trx_Cleaned['Merchant_Name_Cleaned'].unique()

array(['LONG BEACH KING ', 'FAIRPRICE FINEST', 'AGODA.COM INTERNET SG',
       ..., 'DONATION VIA AMMADO DUBLIN IE', 'SAFECART.COM*SAFECART 855',
       'BOTTOMSLIM NEW YORK P '], dtype=object)

In [0]:
Trx_Cleaned.shape, Demographics_Cleaned.shape

((374035, 14), (22582, 21))

In [0]:
def clean(txt):
    exclude = set(string.punctuation)
    freq_term = ['sg','sgp','corp','corporation','inc', 'corporate', 'ltd', 'laboratoires', 'laboratories',
             'brand', 'international','company','gm','ml','com','internet','lte','llc']
    txt = str(txt)    
    txt = ''.join([' ' + x if x.isupper() and i > 0 and txt[i-1] != '-' and i+2 < len(txt) and
                   txt[i+1].islower() and txt[i+2].islower() else x for i, x in enumerate(txt)]).strip() # separate out "HimalayaHerbal" to "Himalaya Herbal" 
    txt=txt.lower()
    txt=''.join([ch for ch in txt if not ch.isdigit()])
    txt = ''.join([ch if ch not in exclude else ' ' for ch in txt])
    txt = ' '.join(x for x in txt.split(' ') if x not in freq_term)
    txt=re.sub(' +',' ',txt)
    return txt.strip()

In [0]:
Trx_Cleaned['Merchant_Name_Cleaned2'] = Trx_Cleaned['Merchant_Name_Cleaned'].apply(lambda x: clean(x))

In [0]:
def fuzzyfind_similar_names(query, choices,score_func, lt, sim_threshold):
  top_similar = []
  sim_scores = process.extract(query, choices, scorer = score_func, limit=lt)
  #print('Y')
  #print(sim_scores)
  for name,score in sim_scores:
    if score > sim_threshold:
      top_similar.append(name)
  return top_similar

In [0]:
print(fuzz.partial_ratio("Starbucks", ["STARBUCKS NZ", "sTARBUCKS LTD"]))
print(fuzz.WRatio("Starbucks", ["STARBUCKS NZ", "sTARBUCKS LTD"]))
print(fuzz.token_sort_ratio("Starbucks", ["STARBUCKS NZ", "sTARBUCKS LTD"]))
print(fuzz.token_set_ratio("Starbucks", ["STARBUCKS NZ", "sTARBUCKS LTD"]))

22
90
51
100


In [0]:
print(fuzz.partial_ratio("Amazon", ["AMAZON web services", "amazon payments", "amazon RETAIL"]))
print(fuzz.WRatio("Amazon", ["AMAZON web services", "amazon payments", "amazon RETAIL"]))
print(fuzz.token_sort_ratio("Amazon", ["AMAZON web services", "amazon payments", "amazon RETAIL"]))
print(fuzz.token_set_ratio("Amazon", ["AMAZON web services", "amazon payments", "amazon RETAIL"]))

83
60
22
100


In [0]:
MerchantList = Trx_Cleaned['Merchant_Name_Cleaned2'].unique()

In [0]:
SimilarNames = {}

### Group into similar names using fuzzywuzzy token set ratio

In [0]:
choices=MerchantList
for index, merchant in enumerate(MerchantList):
  print(index)
  if merchant in choices:
    print(merchant)
    SimilarNames[merchant] = fuzzyfind_similar_names(merchant, choices, score_func = fuzz.token_set_ratio, lt=100, sim_threshold = 80)
    print('SimilarNames:',SimilarNames[merchant])
    print('\n')
    choices= [x for x in choices if x not in SimilarNames[merchant]+[merchant]]

### Creating mapping of merchant group-name to merchant name

In [0]:
# Names_Mapping = pd.DataFrame([(key, var) for (key, L) in SimilarNames.items() for var in L], 
#                  columns=['key', 'variable'])

In [0]:
#Names_Mapping.to_excel('MerchantName_Mapping.xlsx',index = False)

### Manually modified the above merchant name mapping file to create a better quality mapping. Got 14K merchant names. Grouping merchants who together contribute to 10% transaction amount into 'Others' 

In [0]:
names_mapped = pd.read_excel('MerchantName_Mapping_Nanda.xlsx')

In [0]:
names_mapped['key'].nunique()

14914

In [0]:
Trx_Cleaned.columns, names_mapped.columns

(Index(['PartyID', 'CardType', 'Merchant_Category', 'Merchant_Name',
        'Transaction_amount', 'Transaction_Date', 'HasTransacted?',
        'WeekOfYear', 'Month', 'DayOfMonth', 'DayOfWeek', 'Weekday/Weekend',
        'Merchant_Name_Cleaned', 'Merchant_Gateway', 'Merchant_Name_Cleaned2'],
       dtype='object'), Index(['key', 'variable'], dtype='object'))

In [0]:
Trx_Cleaned.shape, names_mapped.shape

((374035, 15), (25295, 2))

In [0]:
Trx_Cleaned = pd.merge(Trx_Cleaned,names_mapped,how = 'left',left_on= 'Merchant_Name_Cleaned2', right_on='variable')

In [0]:
Trx_Cleaned.shape, names_mapped.shape

((374035, 17), (25295, 2))

In [0]:
Trx_Cleaned.rename(columns = {'key':'Merchant_Name_Grouped'},inplace = True)

In [0]:
Trx_Cleaned.drop(columns = 'variable', inplace = True)

In [0]:
MerchantSalesCountPct = pd.DataFrame(Trx_Cleaned['Merchant_Name_Grouped'].value_counts(normalize = True)*100)

In [0]:
MerchantSalesCountPct

,Merchant_Name_Grouped
fairprice,34.359197
esso,2.743034
robinsons,2.620284
caltex upper thomson,1.469530
giosis qoo,1.404277
...,...
pinkplanks,0.000267
hair venture salon,0.000267
dasom,0.000267
siglap optical,0.000267


In [0]:
MerchantSalesAmtPct = Trx_Cleaned.groupby('Merchant_Name_Grouped').agg({'Transaction_amount':'sum'})
MerchantSalesAmtPct

,Transaction_amount
Merchant_Name_Grouped,
7-eleven,4153.80
Aeon Supermarket,11035.30
Air Asia,3558.38
Air Berlin,898.89
Air Canada,11578.49
...,...
zupon beauty slimming c,62.49
zuri express hotel cikarabekasi id,660.41
zurich int life lt douglas gb,6221.60


In [0]:
MerchantSalesAmtPct.sort_values(by='Transaction_amount',  ascending = False, inplace = True)

In [0]:
MerchantSalesAmtPct['PctAmount'] = 100*(MerchantSalesAmtPct['Transaction_amount']/MerchantSalesAmtPct['Transaction_amount'].sum() )
MerchantSalesAmtPct['CumulativePctAmount'] = MerchantSalesAmtPct['PctAmount'].cumsum()
MerchantSalesAmtPct 

,Transaction_amount,PctAmount,CumulativePctAmount
Merchant_Name_Grouped,,,
fairprice,5766136.79,1.144563e+01,11.445630
ntuc income,5341746.43,1.060323e+01,22.048857
robinsons,1652569.39,3.280307e+00,25.329164
esso,732606.05,1.454204e+00,26.783368
life insurance premium,465441.92,9.238900e-01,27.707258
...,...,...,...
jumbo software,1.10,2.183471e-06,99.999996
keepcalling,0.86,1.707077e-06,99.999998
bd acct recovery,0.86,1.707077e-06,99.999999


In [0]:
MerchantSalesPct['Merchant_Name_Grouped'].cumsum()

fairprice                34.359197
esso                     37.102231
robinsons                39.722514
caltex upper thomson     41.192044
giosis qoo               42.596322
                           ...    
pinkplanks               99.998930
hair venture salon       99.999198
dasom                    99.999465
siglap optical           99.999733
dental clini            100.000000
Name: Merchant_Name_Grouped, Length: 14914, dtype: float64

In [0]:
MerchantSalesPct[MerchantSalesPct['Merchant_Name_Grouped'].cumsum()>90].index

Index(['action city', 'beppu menkan rstn s', 'the shilla duty f incheon kr',
       'mst golf', 'jibiru pte', 'nice vision optical', 'china one', 'phspl s',
       'singkids pte', 'soon lee',
       ...
       'love n show pte', 'caba s since taipei city tw', 'hte s pte',
       'iggymcpig', 'internetgam', 'pinkplanks', 'hair venture salon', 'dasom',
       'siglap optical', 'dental clini'],
      dtype='object', length=13559)

In [0]:
MerchantSalesAmtPct[MerchantSalesAmtPct['CumulativePctAmount']>90].index

Index(['viator gb', 'white link', 'tambuah mas indonesian re', 'etna',
       'chrome hearts taipei tw', 'kotobuki restaurant', 'metropolitan ymca',
       'conference online', 'stargek private', 'nana hiso hotel bangkok tha',
       ...
       'digitalchemy', 'distinctdev', 'redblox games', 'javanshir',
       'grigore cretu', 'jumbo software', 'keepcalling', 'bd acct recovery',
       'andrewtong', 'rudding park hotel harrogate gbr'],
      dtype='object', name='Merchant_Name_Grouped', length=12611)

In [0]:
Trx_Cleaned['Merchant_Name_Grouped2'] = Trx_Cleaned['Merchant_Name_Grouped']
Trx_Cleaned.loc[Trx_Cleaned['Merchant_Name_Grouped2'].isin(MerchantSalesAmtPct[MerchantSalesAmtPct['CumulativePctAmount']>90].index),'Merchant_Name_Grouped2'] = 'Others'

In [0]:
Trx_Cleaned['Merchant_Name_Grouped2'].nunique() 

2304

In [0]:
Trx_Cleaned

,PartyID,CardType,Merchant_Category,Merchant_Name,Transaction_amount,Transaction_Date,HasTransacted?,WeekOfYear,Month,DayOfMonth,DayOfWeek,Weekday/Weekend,Merchant_Name_Cleaned,Merchant_Gateway,Merchant_Name_Cleaned2,Merchant_Name_Grouped,Merchant_Name_Grouped2
0,174884,Co-Brand,RESTAURANTS-BARS,LONG BEACH KING SINGAPORE SG,79.43,41699,1,9,3,1,7,Weekend,LONG BEACH KING,NaN,long beach king,long beach king,long beach king
1,174884,Co-Brand,FOOD STORES-WAREHOUSE,FAIRPRICE FINEST-THOMSON SINGAPORE SG,44.50,41705,1,10,3,7,6,Weekday,FAIRPRICE FINEST,NaN,fairprice finest,fairprice,fairprice
2,174884,Co-Brand,TRAVEL AGENCIES,AGODA.COM INTERNET SG,1427.46,41706,1,10,3,8,7,Weekend,AGODA.COM INTERNET SG,NaN,agoda,agoda,agoda
3,174884,Co-Brand,FOOD STORES-WAREHOUSE,FAIRPRICE XTRA-AMK SINGAPORE SG,25.00,41707,1,11,3,9,1,Weekend,FAIRPRICE XTRA,NaN,fairprice xtra,fairprice,fairprice
4,174884,Co-Brand,UTILITIES,HELLO! AMK HUB SINGAPORE SG,308.86,41707,1,11,3,9,1,Weekend,HELLO! AMK HUB,NaN,hello amk hub,hello amk hub,hello amk hub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374030,86556083,Co-Brand,FOOD STORES-WAREHOUSE,NTUC FP-HOUGANG A SINGAPORE SG,53.72,41790,1,22,5,31,7,Weekend,NTUC FP,NaN,ntuc fp,fairprice,fairprice
374031,86556083,Co-Brand,CLOTHING STORES,GIOSIS(QOO10) SINGAPORE SG,5.44,41790,1,22,5,31,7,Weekend,GIOSIS(QOO10),NaN,giosis qoo,giosis qoo,giosis qoo
374032,86607425,Co-Brand,DEPARTMENT STORES,ROBINSONS - RAFFLES CITY SINGAPORE SG,6.44,41790,1,22,5,31,7,Weekend,ROBINSONS,NaN,robinsons,robinsons,robinsons
374033,86607425,Co-Brand,DEPARTMENT STORES,ROBINSONS - RAFFLES CITY SINGAPORE SG,813.96,41790,1,22,5,31,7,Weekend,ROBINSONS,NaN,robinsons,robinsons,robinsons


In [0]:
Trx_Cleaned['Merchant_Category'].unique()

array(['RESTAURANTS-BARS', 'FOOD STORES-WAREHOUSE', 'TRAVEL AGENCIES',
       'UTILITIES', 'MAIL ORDER', 'GAS STATIONS', 'OTHER TRANSPORT',
       'OTHER RETAIL', 'INTERIOR FURNISHINGS', 'VEHICLES', 'HARDWARE',
       'CLOTHING STORES', 'OTHER SERVICES', 'HEALTH CARE', 'RECREATION',
       'DEPARTMENT STORES', 'DRUG STORES', 'ELECTRIC-APPLIANCE',
       'SPORTING-TOY STORES', 'PROFESSIONAL SERVICES', 'AIRLINE',
       'HOTEL-MOTEL', 'EDUCATION', 'AUTO RENTAL', 'QUASI CASH', 'Others',
       'REPAIR SHOPS', 'DISCOUNT STORES'], dtype=object)

In [0]:
#Trx_Cleaned.to_csv('Transactions_Cleaned_Nanda.csv', index = False)

In [0]:
#MerchantSalesAmtPct.to_excel('MerchantSalesAmountPct.xlsx', index = True)